### .mat dosyalarının içeriğini inceleyerek yapıyı anlamak

In [2]:
import os

raw_data_path = "../data/1-raw"
files = os.listdir(raw_data_path)
files

['B0005.mat', 'B0006.mat', 'B0018.mat']

#### bir .mat dosyasını yükleyip yapısını incelemek

In [4]:
import scipy.io as sio

b0005_path = os.path.join(raw_data_path, "B0005.mat")
data = sio.loadmat(b0005_path)
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'B0005'])

In [5]:
battery_data = data["B0005"]
battery_struct = battery_data[0, 0]
battery_struct.dtype

dtype([('cycle', 'O')])

In [6]:
cycles = battery_struct["cycle"][0]
print("Total cycles:", len(cycles))
cycles.dtype

Total cycles: 616


dtype([('type', 'O'), ('ambient_temperature', 'O'), ('time', 'O'), ('data', 'O')])

In [7]:
sample_cycle = cycles[0]

print("Cycle içindeki alanlar:")
for field in sample_cycle.dtype.names:
    print(f" - {field}")

Cycle içindeki alanlar:
 - type
 - ambient_temperature
 - time
 - data


İlk cycle'ın iç yapısını inceleyince her cycle'ın 'type', 'ambient_temperature', 'time' ve 'data' alsnlarını görüyorz. Bu alanların içeriklerine tüm .mat dosyalarıyla toplıca bakalım..

In [9]:
raw_data_path = "../data/1-raw"
mat_files = [f for f in os.listdir(raw_data_path) if f.endswith(".mat")]

for mat_file in mat_files:
    print(f"\n--- {mat_file} ---")

    file_path = os.path.join(raw_data_path, mat_file)
    data = sio.loadmat(file_path)

    key = list(data.keys())[-1]
    battery_data = data[key]
    battery_struct = battery_data[0, 0]
    cycles = battery_struct["cycle"][0]

    # Benzersiz cycle type değerlerini al
    cycle_types = set()
    for cycle in cycles:
        cycle_type = cycle["type"][0]
        cycle_types.add(cycle_type)

    print("Cycle 'type' alanındaki değerler:")
    for t in cycle_types:
        print(" -", t)

    # Her tür için ilk cycle'ı bul ve başlıkları yazdır
    for t in cycle_types:
        for cycle in cycles:
            if cycle["type"][0] == t:
                print(f"\nData '{t}' alanındaki ölçüm başlıkları:")
                data_struct = cycle["data"][0, 0]
                for name in data_struct.dtype.names:
                    print(" -", name)

                # Ambient Temperature
                ambient_temp = cycle["ambient_temperature"][0][0]
                print(f"Ortam sıcaklığı (ambient_temperature): {ambient_temp}")

                # Time
                time_series = cycle["time"][0]
                print(f"Zaman adımı sayısı (time): {len(time_series)}")
                print(f"İlk 5 zaman verisi: {time_series[:5]}")
                break  # sadece ilk örnek yeterli



--- B0005.mat ---
Cycle 'type' alanındaki değerler:
 - impedance
 - discharge
 - charge

Data 'impedance' alanındaki ölçüm başlıkları:
 - Sense_current
 - Battery_current
 - Current_ratio
 - Battery_impedance
 - Rectified_Impedance
 - Re
 - Rct
Ortam sıcaklığı (ambient_temperature): 24
Zaman adımı sayısı (time): 6
İlk 5 zaman verisi: [2008.    4.   18.   20.   55.]

Data 'discharge' alanındaki ölçüm başlıkları:
 - Voltage_measured
 - Current_measured
 - Temperature_measured
 - Current_load
 - Voltage_load
 - Time
 - Capacity
Ortam sıcaklığı (ambient_temperature): 24
Zaman adımı sayısı (time): 6
İlk 5 zaman verisi: [2.008e+03 4.000e+00 2.000e+00 1.500e+01 2.500e+01]

Data 'charge' alanındaki ölçüm başlıkları:
 - Voltage_measured
 - Current_measured
 - Temperature_measured
 - Current_charge
 - Voltage_charge
 - Time
Ortam sıcaklığı (ambient_temperature): 24
Zaman adımı sayısı (time): 6
İlk 5 zaman verisi: [2.008e+03 4.000e+00 2.000e+00 1.300e+01 8.000e+00]

--- B0006.mat ---
Cycle 'type

In [10]:
raw_data_path = "../data/1-raw"
mat_files = [f for f in os.listdir(raw_data_path) if f.endswith(".mat")]

for mat_file in mat_files:
    file_path = os.path.join(raw_data_path, mat_file)
    data = sio.loadmat(file_path)
    key = list(data.keys())[-1]
    battery_data = data[key]
    battery_struct = battery_data[0, 0]
    cycles = battery_struct["cycle"][0]

    type_counts = {"discharge": 0, "charge": 0, "impedance": 0}

    for cycle in cycles:
        cycle_type = cycle["type"][0]
        if cycle_type in type_counts:
            type_counts[cycle_type] += 1

    print(f"\n--- {mat_file} ---")
    for t, count in type_counts.items():
        print(f"{t.capitalize()} veri sayısı: {count}")



--- B0005.mat ---
Discharge veri sayısı: 168
Charge veri sayısı: 170
Impedance veri sayısı: 278

--- B0006.mat ---
Discharge veri sayısı: 168
Charge veri sayısı: 170
Impedance veri sayısı: 278

--- B0018.mat ---
Discharge veri sayısı: 132
Charge veri sayısı: 134
Impedance veri sayısı: 53


Her batarya için farklı cycle türlerinin (discharge, charge, impedance) kaç kez tekrar ettiğini analiz ettik.
Toplam cycle'ların sayıları birbirine yakın oldupu için içerdiği değişkenlere de bakmıştık oradan yola çıkarak ve SoH/SoC formüllerinde ve hesaplamalarında
kullanılabilecek önemli olan "Capacity" değişkeni yalnızca 'discharge' türünde bulunmaktadır. discharge verileri tam ve doğrudan modellemeye uygun formatta olduğu için sadece bu verilerle devam edilecektir.

.mat dosyasındaki veri yapısı çok katmanlı (nested struct) bir yapı içerdiği için doğrudan inceleme işlemleri hem zaman alıcı hem de okunsbilirlik açısından verimsizdir.
Bu nedenle, analiz ve görselleştirme işlemleri için veriyi düzleştirşlmiş ve standartlaştırılmış .csv formatına dönğştürmek çok daha kulanışlıdır.

In [13]:
import pandas as pd
import scipy

def extract_discharge_cycles(mat_file_path, battery_name):
    mat = scipy.io.loadmat(mat_file_path)
    data = mat[battery_name][0][0]
    cycles = data['cycle'][0]
    
    records = []
    discharge_index = 1  # Discharge sırası (1'den başlat)

    for i, cycle in enumerate(cycles):
        if cycle['type'][0] != 'discharge':
            continue

        c_data = cycle['data'][0, 0]

        voltage = c_data['Voltage_measured'][0]
        current = c_data['Current_measured'][0]
        temperature = c_data['Temperature_measured'][0]
        time = c_data['Time'][0]
        if 'Capacity' in c_data.dtype.names:
            cap = c_data['Capacity']
            capacity_array = cap[0] if cap.ndim == 2 else cap
            capacity = capacity_array if len(capacity_array) > 1 else [capacity_array[0]] * len(time)
        else:
            capacity = [None] * len(time)


        for j in range(len(voltage)):
            records.append({
                "battery": battery_name,
                "original_cycle": i,               # .mat içindeki gerçek cycle index
                "discharge_cycle": discharge_index,  # Sıralı discharge index
                "time": time[j],
                "voltage": voltage[j],
                "current": current[j],
                "temperature": temperature[j],
                "capacity": capacity[j]
            })

        discharge_index += 1

    return pd.DataFrame(records)


In [ ]:
def process_all_batteries(mat_files_dir, output_dir, battery_names):
    os.makedirs(output_dir, exist_ok=True)

    for name in battery_names:
        mat_path = os.path.join(mat_files_dir, f"{name}.mat")
        print(f"Processing {name}...")
        df = extract_discharge_cycles(mat_path, name)
        csv_path = os.path.join(output_dir, f"{name}_discharge.csv")
        df.to_csv(csv_path, index=False)
        print(f"Saved to {csv_path}")

battery_files = ['B0005', 'B0006', 'B0018']
output_directory = '../data/2-processed_csv'
process_all_batteries(raw_data_path, output_directory, battery_files)


Processing B0005...
Saved to ../data/2-processed_csv\B0005_discharge.csv
Processing B0006...
Saved to ../data/2-processed_csv\B0006_discharge.csv
Processing B0018...
Saved to ../data/2-processed_csv\B0018_discharge.csv


Tüm batarya dosyaları için yalnızca 'discharge' olan cycle'lar seçilmiştir.
Bu cycle'lardaki zaman ve ölçüm değerleri satır bazında düzleştirilmiş ve 'cycle', 'time', 'ambient_temperature' gibi sabit değişkenlerle birlikte tek bir tabloya yazılmıştır.
Çıktı olarak her batarya için ayrı .csv dosyaları oluşturulmuştur.


Tüm .csv dosyaları tek bir veri kümesinde birleştirerek daha kolay yönetebiliriz. battery sütunu LabelEncoder ile sayısal olarak dönüştürüp ve battery_encoded adıyla veri setine ekleyelim. Bu dosyayı 2. notebook'ta Keşifsel Veri Analizi (EDA) aşamaları için kullanıcam.ç

In [17]:
from sklearn.preprocessing import LabelEncoder

# Klasör yolu ve dosyalar
csv_dir = "../data/2-processed_csv"
files = [f for f in os.listdir(csv_dir) if f.endswith("_discharge.csv")]

# Tüm csv'leri okuyup birleştir
dfs = []
for f in files:
    df = pd.read_csv(os.path.join(csv_dir, f))
    dfs.append(df)

all_data = pd.concat(dfs, ignore_index=True)

# battery kolonunu encode et
le = LabelEncoder()
all_data["battery_encoded"] = le.fit_transform(all_data["battery"])

# Kontrol için eşleştirmeyi yazdır
battery_map = dict(zip(le.classes_, le.transform(le.classes_)))
print("Battery label encoding eşlemesi:", battery_map)

# Kayıt
all_data.to_csv("../data/all_batteries_discharge_combined.csv", index=False)
print("✅ Veriler birleştirildi ve kaydedildi: all_batteries_discharge_combined.csv")


Battery label encoding eşlemesi: {'B0005': 0, 'B0006': 1, 'B0018': 2}
✅ Veriler birleştirildi ve kaydedildi: all_batteries_discharge_combined.csv
